# Estudo preliminar para avaliar a transmissibilidade dos distritos de São Paulo
O presente estudo consiste em servir de inspiração para ideias e ações para tentar nortear ações de reabertura da quarentena ou qualquer ação que ajude no enfrentamento da covid. A partir de dados do IBGE - Censo 2010 e da pesquisa do origem e destino da Companhia Metropolitana de São Paulo de 2017 elaborou-se um  indice que avalia a transmissibilidade em cada distrito.

# Indice de Transmissibilidade

$\textbf{Definição:}$ O indice de transmissibilidade é calculado a partir dos indicadores sociais e indicadores de mobilidade urbano, como por exemplo o perfil de viagens da cidade de São Paulo.

Assim, foram considerados as seguintes variáveis:

* Perfil da distribuição de moradores por dormitório por distrito de São Paulo
* Perfil das ligações de esgoto por distrito de São Paulo
* Perfil da distribuição de Favelas por distrito de São Paulo
* Porcentagem de viagens coletivas motorizadas por distrito de São Paulo
* Total de viagens dentro de um distrito
* Porcentagem de viagens por educação dentro do distrito
* Porcentagem de viagens de médio confinamento do distrito
* Porcentagem de viagens de baixo confinamento do distrito

A expressão que calcula o índice encontra-se abaixo:


$$\begin{equation}
IT = \sum I_{i}*f_{i},
\end{equation}$$

onde $I_i$ corresponde ao atributo dos distritos avaliados e $f_i$ corresponde ao fator de ponderamento desse atributo

In [1]:
import pandas as pd
import numpy as np
import os
import unidecode

def _standardize_the_names(df, col_name='Distrito'):
    df[col_name] = df[col_name].apply(lambda x: unidecode.unidecode(x))
    df[col_name] = df[col_name].apply(lambda x: x.replace('(distrito da capital)', ''))
    df[col_name] = df[col_name].apply(lambda x: x.title())
    df[col_name] = df[col_name].apply(lambda x: x[:-1] if x.endswith(' ') else x).unique()
    return df

In [2]:
#Shapes_files
shapes=pd.read_csv('shapes_sp.csv')
shapes.head()

,Distrito,geometry
0,Jose Bonifacio,POLYGON ((-46.44604214397209 -23.5561703971907...
1,Jardim Sao Luis,POLYGON ((-46.75819187684431 -23.6482539059726...
2,Artur Alvim,POLYGON ((-46.47482722150997 -23.5234761186693...
3,Jaguara,POLYGON ((-46.75581263455317 -23.4948240968001...
4,Vila Prudente,"POLYGON ((-46.58128711243225 -23.572649764946,..."


In [7]:
cols_idx=['Distrito','Idx_total_viagens',
          'Idx_coletivas',
          'Idx_baixo_confinamento',
          'Idx_sem_confinamento','Index_viagens_por_educacao']

#Dados de viagens
mobilidade=_standardize_the_names(pd.read_excel('indices.xlsx',sheet_name='mobilidade'))[cols_idx]

#Dados do perfil domiciliar dos distritos
domicilio=_standardize_the_names(pd.read_excel('indices.xlsx',sheet_name='index domicilio'),col_name='Distritos')

In [8]:
cols=['Distritos',
      'Index domic de moradores por dormitório ponderado', 
      'Index 20% ligado a rede de esgoto', 'Index de domicílios em Favelas']

domicilio_reduzido=domicilio[cols].rename(columns={'Distritos':'Distrito'})
domicilio_reduzido=domicilio_reduzido.apply(lambda x: x/100 if x.name!='Distrito' else x)

In [9]:
#Pesos para dados de mobilidade
fator_coletivo=4 #Transporte coletivo
fator_individual=1 #Transporte individual
fator_medio=2 #Transporte de médio confinamento (Taxi)
fator_educacao=0 #Viagens por motivos de educação que em condicão de quarentena é igual a zero.

In [10]:
mobilidade['Idx_coletivas']=fator_coletivo*mobilidade['Idx_coletivas']
mobilidade['Idx_baixo_confinamento']=fator_medio*mobilidade['Idx_baixo_confinamento']
mobilidade['Idx_sem_confinamento']=fator_individual*mobilidade['Idx_sem_confinamento']
mobilidade['Index_viagens_por_educacao']=fator_educacao*mobilidade['Index_viagens_por_educacao']

In [11]:
df_combinado=pd.merge(mobilidade,domicilio_reduzido,on='Distrito')

In [12]:
df_combinado.set_index('Distrito',inplace=True)

In [13]:
df_final=df_combinado.apply(lambda x: np.sum(x),axis=1).to_frame().reset_index()
df_final.rename(columns={0:'Transmissibilidade'},inplace=True)

In [14]:
transmissibilidade_por_distrito=pd.merge(df_final,shapes,on='Distrito')

In [15]:
transmissibilidade_por_distrito.head()

,Distrito,Transmissibilidade,geometry
0,Agua Rasa,5.006455,"POLYGON ((-46.58128711243225 -23.572649764946,..."
1,Alto De Pinheiros,4.619962,POLYGON ((-46.71921569108098 -23.5363344013688...
2,Anhanguera,6.380384,POLYGON ((-46.78579834173713 -23.4046766027350...
3,Aricanduva,5.394884,POLYGON ((-46.51666242710171 -23.5548090876071...
4,Artur Alvim,5.823862,POLYGON ((-46.47482722150997 -23.5234761186693...
